1. Create a conda python3.8 conda env
`conda create --name snowml python=3.8`

2. You need to install these packages locally
 * peft 
 * transformers


In [ ]:
!pip install /Users/halu/snowml/bazel-bin/snowflake/ml/snowflake_ml_python-1.0.10-py3-none-any.whl

In [5]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
from snowflake.snowpark import Session
from snowflake.ml.utils.connection_params import SnowflakeLoginOptions

Connection config available at ~/.snowsql/config

In [10]:
session = Session.builder.configs(SnowflakeLoginOptions('connections.demo')).create()

SnowflakeLoginOptions() is in private preview since 0.2.0. Do not use it in production. 


In [12]:
session.get_current_database(), session.get_current_schema()

('"HALU_FT"', '"PUBLIC"')

In [13]:
REGISTRY_DATABASE_NAME = "HALU_MR"
REGISTRY_SCHEMA_NAME = "PUBLIC"

In [14]:
from snowflake.ml.registry import model_registry

model_registry.create_model_registry(
    session=session, database_name=REGISTRY_DATABASE_NAME, schema_name=REGISTRY_SCHEMA_NAME
)
registry = model_registry.ModelRegistry(
    session=session, database_name=REGISTRY_DATABASE_NAME, schema_name=REGISTRY_SCHEMA_NAME
)

In [15]:
from snowflake.ml.model.models import llm

In [17]:
options = llm.LLMOptions(token="....")
model = llm.LLM(
    model_id_or_path="/Users/halu/Downloads/halu_peft_ft",
    options=options
)

In [18]:
svc_model = registry.log_model(
    model_name='halu_ft_model_1',
    model_version='v1',
    model=model,
    options={"embed_local_ml_library": True},
)

In [27]:
from snowflake.ml.model import deploy_platforms

deployment_options = {
    "compute_pool": 'BUILD_2023_POOL',
    "num_gpus": 1,
    "image_repo": 'sfengineering-servicesnow.registry.snowflakecomputing.com/halu_ft_db/public/haul_repo',
    "enable_remote_image_build": True,
    "model_in_image": True,
}
        
deploy_info = svc_model.deploy(
    deployment_name="halu_ft_deploy_1",
    platform=deploy_platforms.TargetPlatform.SNOWPARK_CONTAINER_SERVICES,
    permanent=True,
    options=deployment_options
)
deploy_info

{'name': 'HALU_MR.PUBLIC.halu_ft_deploy_1',
 'platform': <TargetPlatform.SNOWPARK_CONTAINER_SERVICES: 'SNOWPARK_CONTAINER_SERVICES'>,
 'target_method': 'infer',
 'signature': ModelSignature(
                     inputs=[
                         FeatureSpec(dtype=DataType.STRING, name='input')
                     ],
                     outputs=[
                         FeatureSpec(dtype=DataType.STRING, name='generated_text')
                     ]
                 ),
 'options': {'compute_pool': 'BUILD_2023_POOL',
  'num_gpus': 1,
  'image_repo': 'sfengineering-servicesnow.registry.snowflakecomputing.com/halu_ft_db/public/haul_repo',
  'enable_remote_image_build': True,
  'model_in_image': True},
 'details': {'image_name': 'sfengineering-servicesnow.registry.snowflakecomputing.com/halu_ft_db/public/haul_repo/4b7980a43a1ff656d23b9401e4471bcd4f021d39:latest',
  'service_spec': 'spec:\n  container:\n  - env:\n      NUM_WORKERS: 1\n      SNOWML_USE_GPU: true\n      TARGET_METHOD: infer

In [28]:
sample = """
[INST] <<SYS>>
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
<</SYS>>
### Instruction:
Extract JSON response with 'location' and 'toy_list' as keys.
'location': Location string of the caller.
'toy_list": List of toy names from the caller.
### Input:
    "frosty: Hello, good friend! You're talking to Frosty! What's your name?
caller: My name's Oliver. And I'm calling from Perth.
frosty: Nice to meet you, Oliver from Perth! So, what's on your wish list this year?
caller: I want a mickey, please.
frosty: Look forward to some Mickey adventures!"
[/INST]
"""

In [24]:
import pandas as pd

In [29]:
input_df = pd.DataFrame({'input': [sample, sample, sample]})

In [30]:
res = svc_model.predict(
    deployment_name='halu_ft_deploy_1',
    data=input_df
)

In [32]:
pd.set_option('display.max_colwidth', None)

In [33]:
res

,generated_text
0,"{""toy_list"": [""Fisher-Price Little People Mickey and Friends""], ""location"": ""Perth""}"
1,"{""toy_list"": [""Fisher-Price Little People Mickey and Friends""], ""location"": ""Perth""}"
2,"{""toy_list"": [""Fisher-Price Little People Mickey and Friends""], ""location"": ""Perth""}"
